<img align="right" src="https://github.com/eo2cube/eo2cube_book/blob/7880672deff906b41f993c856fe1a7eb38ed5b3a/images/banner_siegel.png?raw=true" style="width:1000px;">

# Xarray-I: Data Structure 

## Background

The Python library **`xarray`** is the form in which earth observation data are usually stored in a datacube.
It is an open source project and Python package which offers a toolkit for working with ***multi-dimensional arrays*** of data. **`xarray.dataset`** is an in-memory representation of a netCDF (network Common Data Form) file. Understanding the structure of a **`xarray.dataset`** is the key to enabling our work with these data. Thus, in this notebook, we are mainly dedicated to helping users of our datacube understand its data structure.

## Description

In this notebook, topics covered include:
* **What is inside a `xrray.dataset` (the structure)?**
* **(Basic) Subset Dataset / DataArray**
* **Reshape a Dataset**

In [ ]:
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt

from pystac_client import Client
import planetary_computer as pc
from odc.stac import stac_load

# Set config for displaying tables nicely
pd.set_option("display.max_colwidth", 200)
pd.set_option("display.max_rows", None)


In [ ]:
# Load data from Planetary Computer (STAC)
STAC_URL = "https://planetarycomputer.microsoft.com/api/stac/v1"
COLLECTION = "sentinel-2-l2a"

# Area of interest: Würzburg (EPSG:4326)
bbox = (9.88, 49.75, 10.00, 49.82)

# Output grid
crs = "EPSG:32632"
resolution = 20

# Search STAC items
catalog = Client.open(STAC_URL)

datetime = "2021-03-01/2021-06-15"
query = {"eo:cloud_cover": {"lt": 40}}

search = catalog.search(collections=[COLLECTION], bbox=bbox, datetime=datetime, query=query)
items = list(search.get_items())
len(items)

# Load pixels with odc-stac
bands = ["B02", "B03", "B04"]
resampling = {"*": "bilinear"}

ds_raw = stac_load(
    items,
    bands=bands,
    crs=crs,
    resolution=resolution,
    groupby="solar_day",
    patch_url=pc.sign,
    dtype="uint16",
    nodata=0,
    resampling=resampling,
)

# Rename to match the variable names used throughout this notebook
rename_map = {"B02": "blue", "B03": "green", "B04": "red"}
ds = ds_raw.rename({k: v for k, v in rename_map.items() if k in ds_raw.data_vars})

# Scale reflectance (Sentinel-2 L2A) to ~0..1
for name in list(ds.data_vars):
    ds[name] = ds[name].astype("float32") * 1e-4

ds


In [24]:
da = ds.to_array().rename({"variable":"band"})
print(da)

<xarray.DataArray (band: 3, time: 24, y: 905, x: 977)>
array([[[[ 9640,  9624,  9536, ...,  9928,  9968,  9992],
         [ 9576,  9592,  9568, ...,  9928,  9920, 10000],
         [ 9648,  9576,  9600, ...,  9928,  9960, 10016],
         ...,
         [ 7676,  7612,  7636, ...,  9096,  9096,  9128],
         [ 7708,  7612,  7704, ...,  9040,  9096,  9056],
         [ 7676,  7568,  7656, ...,  9056,  9040,  9032]],

        [[ 2156,  1876,  1705, ...,  1531,  1570,  1626],
         [ 1700,  1602,  1626, ...,  1456,  1520,  1624],
         [ 2274,  2714,  1968, ...,  1417,  1475,  1598],
         ...,
         [ 1206,  1208,  1220, ...,  1292,  1317,  1290],
         [ 1191,  1189,  1174, ...,  1315,  1313,  1308],
         [ 1226,  1188,  1151, ...,  1323,  1291,  1338]],

        [[ 7208,  7468,  7676, ...,  9392,  9472,  9632],
         [ 6836,  7208,  7280, ...,  9400,  9424,  9512],
         [ 6180,  6556,  6684, ...,  9360,  9416,  9464],
         ...,
...
         ...,
         [ 

In [25]:
ds2 = da.to_dataset(dim="time")
ds2

<xarray.Dataset>
Dimensions:                     (band: 3, y: 905, x: 977)
Coordinates:
  * y                           (y) float64 1.558e+07 1.558e+07 ... 1.557e+07
  * x                           (x) float64 -3.002e+05 -3.002e+05 ... -2.904e+05
    spatial_ref                 int32 32734
  * band                        (band) object 'blue' 'green' 'red'
Data variables: (12/24)
    2022-03-02 10:19:41.024000  (band, y, x) uint16 9640 9624 9536 ... 8108 8056
    2022-03-05 10:29:21.024000  (band, y, x) uint16 2156 1876 1705 ... 1520 1541
    2022-03-07 10:17:59.024000  (band, y, x) uint16 7208 7468 7676 ... 7280 7416
    2022-03-10 10:27:39.025000  (band, y, x) uint16 2118 1904 1750 ... 1479 1491
    2022-03-12 10:18:31.024000  (band, y, x) uint16 2346 2050 1885 ... 1581 1640
    2022-03-15 10:28:11.025000  (band, y, x) uint16 7172 7176 7172 ... 6640 6688
    ...                          ...
    2022-04-16 10:15:59.024000  (band, y, x) uint16 2160 1919 1664 ... 1590 1610
    2022-04-21 10:16:01.025000  (band, y, x) uint16 3944 3636 3386 ... 2356 2344
    2022-04-24 10:30:31.024000  (band, y, x) uint16 7564 7356 7372 ... 8188 8192
    2022-04-26 10:15:49.025000  (band, y, x) uint16 10920 10960 ... 9624 9672
    2022-04-29 10:25:49.024000  (band, y, x) uint16 3890 3774 3752 ... 2266 2290
    2022-05-01 10:16:01.024000  (band, y, x) uint16 7940 7636 7260 ... 9824 9352
Attributes:
    crs:           EPSG:32734
    grid_mapping:  spatial_ref

## **What is inside a `xarray.dataset`?**
The figure below is a diagram depicting the structure of the **`xarray.dataset`** we've just loaded. We hope you may better interpret the texts below explaining the data structure of a **`xarray.dataset`**, with the diagram.

![xarray data structure](https://live.staticflickr.com/65535/51083605166_70dd29baa8_k.jpg)

As we read from the output block, this dataset has three ***Data Variables***, "blue", "green", and "red" (shown with colors in the diagram), referring to the individual spectral band.

Each data variable can be regarded as a **multi-dimensional *Data Array*** with the same structure. It is a **three-dimensional array** (shown as a 3D cube in the diagram) where `time`, `x`, and `y` are its ***Dimensions*** (shown as the axis along with each cube in the diagram).

In this dataset, there are 49 ***coordinates*** under the `time` dimension, which means there are 49 time steps along the `time` axis. There are 1010 coordinates under `x` dimension and 1031 coordinates under `y` dimension, indicating 1010 pixels along `x` axis and 1031 pixels along `y` axis.

The term ***dataset*** is like a *container* holding all the multi-dimensional arrays of the same structure (shown as the red-lined box containing all 3D Cubes in the diagram).

So this instance dataset has a spatial extent of 1010 by 1031 pixels at given long/lat locations, spans over 49 time stamps and includes 3 spectral band.

**In summary, *`xarray.dataset`* is substantially a container for high-dimensional *`DataArray`* with common attributes (e.g., crs) attached:**
* **Data Variables (`values`)**: It's generally the first/highest dimension to subset from a high dimensional array. Each `data variable` contains a multi-dimensional array of all other dimensions.
* **Dimensions (`dims`)**: Other dimensions arranged in hierachical order *(e.g. 'time', 'y', 'x')*.
* **Coordinates (`coords`)**: Coordinates along each `Dimension` *(e.g. timesteps along 'time' dimension, latitudes along 'y' dimension, longitudes along 'x' dimension)*
* **Attributes (`attrs`)**: A dictionary(`dict`) containing Metadata.

Now let's deconstruct the dataset we have just loaded a bit further to have things more clarified!:D

* **To check the structure of the dataset**

In [26]:
ds.values

<bound method Mapping.values of <xarray.Dataset>
Dimensions:      (time: 24, y: 905, x: 977)
Coordinates:
  * time         (time) datetime64[ns] 2022-03-02T10:19:41.024000 ... 2022-05...
  * y            (y) float64 1.558e+07 1.558e+07 ... 1.557e+07 1.557e+07
  * x            (x) float64 -3.002e+05 -3.002e+05 ... -2.905e+05 -2.904e+05
    spatial_ref  int32 32734
Data variables:
    blue         (time, y, x) uint16 9640 9624 9536 9552 ... 12576 12312 11480
    green        (time, y, x) uint16 8992 8872 8896 8960 ... 11080 10680 10024
    red          (time, y, x) uint16 8488 8448 8408 8416 ... 10192 9824 9352
Attributes:
    crs:           EPSG:32734
    grid_mapping:  spatial_ref>

* **To check existing dimensions of the dataset**

In [27]:
ds.dims

Frozen({'time': 24, 'y': 905, 'x': 977})

* **To check the coordinates of the dataset**

In [28]:
ds.coords

Coordinates:
  * time         (time) datetime64[ns] 2022-03-02T10:19:41.024000 ... 2022-05...
  * y            (y) float64 1.558e+07 1.558e+07 ... 1.557e+07 1.557e+07
  * x            (x) float64 -3.002e+05 -3.002e+05 ... -2.905e+05 -2.904e+05
    spatial_ref  int32 32734

* **To check all coordinates along a specific dimension**
<br>
<img src=https://live.staticflickr.com/65535/51115452191_ec160d4514_o.png, width="450">

In [29]:
ds.time
# OR
#ds.coords['time']

<xarray.DataArray 'time' (time: 24)>
array(['2022-03-02T10:19:41.024000000', '2022-03-05T10:29:21.024000000',
       '2022-03-07T10:17:59.024000000', '2022-03-10T10:27:39.025000000',
       '2022-03-12T10:18:31.024000000', '2022-03-15T10:28:11.025000000',
       '2022-03-17T10:16:49.024000000', '2022-03-20T10:26:39.024000000',
       '2022-03-22T10:17:11.024000000', '2022-03-25T10:26:51.024000000',
       '2022-03-27T10:16:29.024000000', '2022-03-30T10:26:19.024000000',
       '2022-04-01T10:16:01.024000000', '2022-04-04T10:26:01.024000000',
       '2022-04-06T10:15:59.025000000', '2022-04-09T10:25:59.024000000',
       '2022-04-11T10:16:01.024000000', '2022-04-14T10:26:01.024000000',
       '2022-04-16T10:15:59.024000000', '2022-04-21T10:16:01.025000000',
       '2022-04-24T10:30:31.024000000', '2022-04-26T10:15:49.025000000',
       '2022-04-29T10:25:49.024000000', '2022-05-01T10:16:01.024000000'],
      dtype='datetime64[ns]')
Coordinates:
  * time         (time) datetime64[ns] 2022-03-02T10:19:41.024000 ... 2022-05...
    spatial_ref  int32 32734
Attributes:
    units:    seconds since 1970-01-01 00:00:00

* **To check attributes of the dataset**

In [30]:
ds.attrs

{'crs': 'EPSG:32734', 'grid_mapping': 'spatial_ref'}

## **Subset Dataset / DataArray**

* **To select all data of "blue" band**
<br>
<img src=https://live.staticflickr.com/65535/51115092614_366cb774a8_o.png, width="350">

In [31]:
ds.blue
# OR
#ds['blue']

<xarray.DataArray 'blue' (time: 24, y: 905, x: 977)>
array([[[ 9640,  9624,  9536, ...,  9928,  9968,  9992],
        [ 9576,  9592,  9568, ...,  9928,  9920, 10000],
        [ 9648,  9576,  9600, ...,  9928,  9960, 10016],
        ...,
        [ 7676,  7612,  7636, ...,  9096,  9096,  9128],
        [ 7708,  7612,  7704, ...,  9040,  9096,  9056],
        [ 7676,  7568,  7656, ...,  9056,  9040,  9032]],

       [[ 2156,  1876,  1705, ...,  1531,  1570,  1626],
        [ 1700,  1602,  1626, ...,  1456,  1520,  1624],
        [ 2274,  2714,  1968, ...,  1417,  1475,  1598],
        ...,
        [ 1206,  1208,  1220, ...,  1292,  1317,  1290],
        [ 1191,  1189,  1174, ...,  1315,  1313,  1308],
        [ 1226,  1188,  1151, ...,  1323,  1291,  1338]],

       [[ 7208,  7468,  7676, ...,  9392,  9472,  9632],
        [ 6836,  7208,  7280, ...,  9400,  9424,  9512],
        [ 6180,  6556,  6684, ...,  9360,  9416,  9464],
        ...,
...
        ...,
        [ 9688,  9664,  9592, ..., 10064,  9992,  9992],
        [ 9664,  9640,  9680, ..., 10064, 10008, 10008],
        [ 9656,  9664,  9616, ...,  9992, 10000, 10088]],

       [[ 3890,  3774,  3752, ...,  3604,  3612,  3648],
        [ 3876,  3786,  3756, ...,  3612,  3598,  3578],
        [ 3844,  3910,  3774, ...,  3564,  3562,  3554],
        ...,
        [ 2011,  1984,  1983, ...,  2468,  2476,  2498],
        [ 2030,  2006,  1966, ...,  2450,  2476,  2462],
        [ 2004,  1974,  1958, ...,  2478,  2474,  2500]],

       [[ 7940,  7636,  7260, ..., 12000, 12192, 12320],
        [ 7768,  7396,  6540, ..., 12168, 12680, 12944],
        [ 7120,  6444,  5628, ..., 12936, 13120, 13304],
        ...,
        [17248, 17248, 17200, ..., 12344, 12032, 12064],
        [17280, 17248, 17184, ..., 12712, 12504, 11808],
        [17376, 17280, 17120, ..., 12576, 12312, 11480]]], dtype=uint16)
Coordinates:
  * time         (time) datetime64[ns] 2022-03-02T10:19:41.024000 ... 2022-05...
  * y            (y) float64 1.558e+07 1.558e+07 ... 1.557e+07 1.557e+07
  * x            (x) float64 -3.002e+05 -3.002e+05 ... -2.905e+05 -2.904e+05
    spatial_ref  int32 32734
Attributes:
    units:         1
    nodata:        0
    crs:           EPSG:32734
    grid_mapping:  spatial_ref

In [18]:
# Only print pixel values
ds.blue.values

array([[[1122, 1094, 1082, ...,  834,  839,  875],
        [1118, 1080, 1108, ...,  835,  839,  846],
        [1108, 1098, 1112, ...,  789,  828,  821],
        ...,
        [ 945,  944,  987, ...,  683,  772,  922],
        [ 982, 1036, 1042, ...,  658,  727,  866],
        [ 982, 1070, 1096, ...,  622,  700,  880]],

       [[1112, 1096, 1074, ...,    0,    0,    0],
        [1110, 1074, 1090, ...,    0,    0,    0],
        [1080, 1050, 1072, ...,    0,    0,    0],
        ...,
        [   0,    0,    0, ...,    0,    0,    0],
        [   0,    0,    0, ...,    0,    0,    0],
        [   0,    0,    0, ...,    0,    0,    0]],

       [[7328, 7304, 7296, ..., 9064, 8952, 8824],
        [7368, 7296, 7296, ..., 9000, 8888, 8704],
        [7364, 7340, 7328, ..., 9000, 8920, 8760],
        ...,
        [6608, 6588, 6592, ..., 7208, 7216, 7184],
        [6596, 6644, 6620, ..., 7252, 7228, 7216],
        [6608, 6632, 6620, ..., 7260, 7220, 7252]],

       ...,

       [[ 465,  449,  44

* **To select blue band data at the first time stamp**
<br>
<img src=https://live.staticflickr.com/65535/51116131265_8464728bc1_o.png, width="350">

In [32]:
ds.blue[0]

<xarray.DataArray 'blue' (y: 905, x: 977)>
array([[ 9640,  9624,  9536, ...,  9928,  9968,  9992],
       [ 9576,  9592,  9568, ...,  9928,  9920, 10000],
       [ 9648,  9576,  9600, ...,  9928,  9960, 10016],
       ...,
       [ 7676,  7612,  7636, ...,  9096,  9096,  9128],
       [ 7708,  7612,  7704, ...,  9040,  9096,  9056],
       [ 7676,  7568,  7656, ...,  9056,  9040,  9032]], dtype=uint16)
Coordinates:
    time         datetime64[ns] 2022-03-02T10:19:41.024000
  * y            (y) float64 1.558e+07 1.558e+07 ... 1.557e+07 1.557e+07
  * x            (x) float64 -3.002e+05 -3.002e+05 ... -2.905e+05 -2.904e+05
    spatial_ref  int32 32734
Attributes:
    units:         1
    nodata:        0
    crs:           EPSG:32734
    grid_mapping:  spatial_ref

* **To select blue band data at the first time stamp while the latitude is the largest in the defined spatial extent**
<img src=https://live.staticflickr.com/65535/51115337046_aeb75d0d03_o.png, width="350">

In [33]:
ds.blue[0][0]

<xarray.DataArray 'blue' (x: 977)>
array([ 9640,  9624,  9536,  9552,  9480,  9456,  9520,  9488,  9456,
        9336,  9304,  9248,  9424,  9368,  9424,  9504,  9520,  9552,
        9664,  9696,  9712,  9696,  9688,  9696,  9712,  9760,  9768,
        9744,  9680,  9664,  9704,  9712,  9704,  9768,  9760,  9792,
        9752,  9760,  9744,  9752,  9824,  9768,  9784,  9776,  9792,
        9832,  9864,  9816,  9816,  9816,  9856,  9816,  9768,  9704,
        9776,  9792,  9704,  9744,  9736,  9752,  9704,  9664,  9664,
        9664,  9640,  9568,  9560,  9568,  9424,  9368,  9368,  9440,
        9360,  9344,  9296,  9232,  9152,  9192,  9152,  9168,  9080,
        9128,  9056,  9032,  9120,  9072,  9056,  9144,  9176,  9136,
        9136,  9168,  9136,  9152,  9168,  9200,  9176,  9216,  9192,
        9192,  9168,  9264,  9264,  9312,  9336,  9360,  9384,  9400,
        9424,  9480,  9472,  9480,  9536,  9576,  9568,  9568,  9560,
        9568,  9568,  9568,  9600,  9568,  9592,  9600,  9616,  9648,
        9616,  9664,  9744,  9744,  9760,  9728,  9744,  9792,  9792,
        9744,  9768,  9792,  9728,  9672,  9672,  9656,  9608,  9640,
        9616,  9640,  9576,  9512,  9448,  9512,  9528,  9528,  9480,
        9544,  9464,  9576,  9664,  9640,  9664,  9680,  9680,  9616,
        9616,  9512,  9560,  9528,  9544,  9568,  9568,  9512,  9560,
        9600,  9560,  9608,  9608,  9616,  9592,  9704,  9616,  9624,
...
       10224, 10368, 10368, 10312, 10240, 10224, 10256, 10264, 10224,
       10280, 10280, 10224, 10192, 10112, 10144, 10048, 10000,  9872,
        9888,  9872,  9792,  9688,  9688,  9616,  9576,  9576,  9488,
        9520,  9512,  9488,  9416,  9520,  9560,  9480,  9464,  9456,
        9392,  9440,  9432,  9504,  9552,  9584,  9712,  9768,  9768,
        9760,  9784,  9800,  9752,  9840,  9856,  9856,  9776,  9856,
        9792,  9840,  9824,  9792,  9712,  9728,  9688,  9696,  9696,
        9616,  9592,  9576,  9560,  9520,  9560,  9544,  9520,  9544,
        9560,  9544,  9616,  9632,  9664,  9664,  9648,  9712,  9728,
        9688,  9792,  9696,  9696,  9696,  9688,  9712,  9712,  9624,
        9680,  9728,  9760,  9704,  9552,  9552,  9528,  9504,  9464,
        9568,  9568,  9552,  9528,  9496,  9528,  9552,  9496,  9480,
        9440,  9328,  9344,  9344,  9328,  9312,  9304,  9152,  9128,
        9144,  9128,  9080,  9016,  9032,  8992,  8992,  8928,  8904,
        8976,  8944,  9016,  8976,  8984,  9040,  9032,  9120,  9104,
        9144,  9208,  9224,  9304,  9368,  9464,  9504,  9456,  9488,
        9512,  9560,  9640,  9640,  9624,  9624,  9656,  9624,  9680,
        9736,  9712,  9744,  9816,  9848,  9856,  9848,  9928,  9872,
        9848,  9880,  9904,  9904,  9944,  9928,  9880,  9872,  9856,
        9840,  9880,  9928,  9968,  9992], dtype=uint16)
Coordinates:
    time         datetime64[ns] 2022-03-02T10:19:41.024000
    y            float64 1.558e+07
  * x            (x) float64 -3.002e+05 -3.002e+05 ... -2.905e+05 -2.904e+05
    spatial_ref  int32 32734
Attributes:
    units:         1
    nodata:        0
    crs:           EPSG:32734
    grid_mapping:  spatial_ref

* **To select the upper-left corner pixel**
<br>
<img src=https://live.staticflickr.com/65535/51116131235_b0cca9589f_o.png, width="350">

In [34]:
ds.blue[0][0][0]

<xarray.DataArray 'blue' ()>
array(9640, dtype=uint16)
Coordinates:
    time         datetime64[ns] 2022-03-02T10:19:41.024000
    y            float64 1.558e+07
    x            float64 -3.002e+05
    spatial_ref  int32 32734
Attributes:
    units:         1
    nodata:        0
    crs:           EPSG:32734
    grid_mapping:  spatial_ref

### **subset dataset with `isel` vs. `sel`**
* Use `isel` when subsetting with **index**
* Use `sel` when subsetting with **labels**

* **To select data of all spectral bands at the first time stamp**
<br>
<img src=https://live.staticflickr.com/65535/51114879732_7d62db54f4_o.png, width="750">

In [35]:
ds.isel(time=[0])

<xarray.Dataset>
Dimensions:      (time: 1, y: 905, x: 977)
Coordinates:
  * time         (time) datetime64[ns] 2022-03-02T10:19:41.024000
  * y            (y) float64 1.558e+07 1.558e+07 ... 1.557e+07 1.557e+07
  * x            (x) float64 -3.002e+05 -3.002e+05 ... -2.905e+05 -2.904e+05
    spatial_ref  int32 32734
Data variables:
    blue         (time, y, x) uint16 9640 9624 9536 9552 ... 8992 9056 9040 9032
    green        (time, y, x) uint16 8992 8872 8896 8960 ... 8440 8432 8416 8456
    red          (time, y, x) uint16 8488 8448 8408 8416 ... 8152 8080 8108 8056
Attributes:
    crs:           EPSG:32734
    grid_mapping:  spatial_ref

* **To select data of all spectral bands of year 2020** 
<br>
<img src=https://live.staticflickr.com/65535/51116281070_75f1b46a9c_o.png, width="750">

In [40]:
ds.sel(time='2021-03-02')

<xarray.Dataset>
Dimensions:      (time: 1, y: 905, x: 977)
Coordinates:
  * time         (time) datetime64[ns] 2021-03-02T10:18:39.025000
  * y            (y) float64 1.558e+07 1.558e+07 ... 1.557e+07 1.557e+07
  * x            (x) float64 -3.002e+05 -3.002e+05 ... -2.905e+05 -2.904e+05
    spatial_ref  int32 32734
Data variables:
    blue         (time, y, x) uint16 1938 1691 1581 1399 ... 1353 1376 1365 1378
    green        (time, y, x) uint16 2118 1872 1770 1514 ... 1419 1412 1437 1413
    red          (time, y, x) uint16 2274 2032 1958 1564 ... 1514 1514 1512 1483
Attributes:
    crs:           EPSG:32734
    grid_mapping:  spatial_ref

## **Reshape Dataset**

* **Convert the Dataset (subset to 2022-03) to a *4-dimension* DataArray**

In [41]:
da = ds.sel(time='2021-03').to_array().rename({"variable":"band"})
da

<xarray.DataArray (band: 3, time: 12, y: 905, x: 977)>
array([[[[ 1938,  1691,  1581, ...,  1546,  1575,  1530],
         [ 2378,  2024,  1842, ...,  1525,  1576,  1586],
         [ 1980,  1744,  1748, ...,  1499,  1510,  1570],
         ...,
         [ 1290,  1286,  1282, ...,  1333,  1400,  1369],
         [ 1283,  1286,  1269, ...,  1331,  1394,  1380],
         [ 1316,  1306,  1276, ...,  1376,  1365,  1378]],

        [[ 8352,  8312,  8304, ...,  6308,  6268,  6328],
         [ 8140,  7996,  7972, ...,  6132,  6068,  6004],
         [ 8028,  7932,  7888, ...,  6124,  6064,  6036],
         ...,
         [10144, 10232, 10296, ...,  8328,  8056,  7776],
         [10296, 10392, 10496, ...,  8432,  8216,  7896],
         [10568, 10576, 10624, ...,  8400,  8240,  7828]],

        [[ 1940,  1739,  1728, ...,  1447,  1475,  1458],
         [ 1824,  1752,  1684, ...,  1363,  1398,  1480],
         [ 1930,  2188,  1848, ...,  1345,  1403,  1476],
         ...,
...
         ...,
         [ 2028,  2062,  2058, ...,  1886,  1864,  1870],
         [ 2050,  2024,  2026, ...,  1886,  1870,  1863],
         [ 2034,  2018,  1986, ...,  1847,  1878,  1870]],

        [[ 8124,  8488,  8784, ...,  1736,  1814,  1938],
         [ 7984,  8392,  8624, ...,  1605,  1722,  1920],
         [ 7744,  8184,  8368, ...,  1579,  1708,  1845],
         ...,
         [ 5224,  5488,  5696, ...,  1314,  1322,  1312],
         [ 5476,  5708,  5948, ...,  1316,  1314,  1300],
         [ 5932,  6136,  6224, ...,  1314,  1323,  1318]],

        [[ 2570,  2296,  2170, ...,  1638,  1716,  1747],
         [ 2126,  2016,  2064, ...,  1530,  1606,  1722],
         [ 2924,  3282,  2490, ...,  1482,  1542,  1612],
         ...,
         [ 1562,  1545,  1478, ...,  1594,  1557,  1543],
         [ 1546,  1502,  1409, ...,  1604,  1580,  1564],
         [ 1504,  1468,  1348, ...,  1524,  1524,  1546]]]], dtype=uint16)
Coordinates:
  * time         (time) datetime64[ns] 2021-03-02T10:18:39.025000 ... 2021-03...
  * y            (y) float64 1.558e+07 1.558e+07 ... 1.557e+07 1.557e+07
  * x            (x) float64 -3.002e+05 -3.002e+05 ... -2.905e+05 -2.904e+05
    spatial_ref  int32 32734
  * band         (band) object 'blue' 'green' 'red'
Attributes:
    crs:           EPSG:32734
    grid_mapping:  spatial_ref

* **Convert the *4-dimension* DataArray back to a Dataset by setting the "time" as DataVariable (reshaped)**

![ds_reshaped](https://live.staticflickr.com/65535/51151694092_ca550152d6_o.png)

In [38]:
ds_reshp = da.to_dataset(dim="time")
print(ds_reshp)

<xarray.Dataset>
Dimensions:                     (band: 3, y: 905, x: 977)
Coordinates:
  * y                           (y) float64 1.558e+07 1.558e+07 ... 1.557e+07
  * x                           (x) float64 -3.002e+05 -3.002e+05 ... -2.904e+05
    spatial_ref                 int32 32734
  * band                        (band) object 'blue' 'green' 'red'
Data variables:
    2022-03-02 10:19:41.024000  (band, y, x) uint16 9640 9624 9536 ... 8108 8056
    2022-03-05 10:29:21.024000  (band, y, x) uint16 2156 1876 1705 ... 1520 1541
    2022-03-07 10:17:59.024000  (band, y, x) uint16 7208 7468 7676 ... 7280 7416
    2022-03-10 10:27:39.025000  (band, y, x) uint16 2118 1904 1750 ... 1479 1491
    2022-03-12 10:18:31.024000  (band, y, x) uint16 2346 2050 1885 ... 1581 1640
    2022-03-15 10:28:11.025000  (band, y, x) uint16 7172 7176 7172 ... 6640 6688
    2022-03-17 10:16:49.024000  (band, y, x) uint16 6024 6016 6040 ... 6724 6640
    2022-03-20 10:26:39.024000  (band, y, x) uint16 2040 18

***
## Additional information

This notebook is for the usage of Jupyter Notebook of the [Department of Remote Sensing](http://remote-sensing.org/), [University of Wuerzburg](https://www.uni-wuerzburg.de/startseite/).

**License:** The code in this notebook is licensed under the [Apache License, Version 2.0](https://www.apache.org/licenses/LICENSE-2.0). 


**Contact:** If you would like to report an issue with this notebook, you can file one on [Github](https://github.com).

**Last modified:** December 2023